First, we'll import pytorch and check if a GPU is available.

In [1]:
import torch
import random
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from FrEIA.modules import *
from FrEIA.framework import *
from pathlib import Path
from torch.utils.data import Dataset, DataLoader

if torch.cuda.is_available():
    print('GPU available')
else:
    print('CPU only')

use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True

CPU only


Next, define the path to the data we're using

In [2]:
data_path = Path('C:\\Users\\dohert01\\PycharmProjects\\qPAI_cINN_uncertainty_estimation\\datasets')
experiment_name = "FlowPhantom_insilico_complicated"

Let's have a look at the data. First of all, borrow some normalisation functions...

In [3]:
def spectrum_normalisation(spectrum):
    """Applies z-score scaling to the initial pressure spectrum"""
    mean = np.mean(spectrum)
    std = np.std(spectrum)
    norm = (spectrum - mean)/std
    return norm

def spectrum_processing(spectrum, allowed_datapoints):
    """Returns a normalised initial pressure spectrum with some of the values zeroed out"""
    num_non_zero_datapoints = random.choice(allowed_datapoints)
    a = np.zeros(len(spectrum))
    a[:num_non_zero_datapoints] = 1
    np.random.shuffle(a)

    incomplete_spectrum = list(np.multiply(a, np.array(spectrum)))
    non_zero_indices = np.nonzero(incomplete_spectrum)
    non_zero_values = list(filter(None,incomplete_spectrum))
    normalised_non_zero = spectrum_normalisation(non_zero_values)

    i = 0
    for index in non_zero_indices[0]:
        incomplete_spectrum[index] = normalised_non_zero[i]
        i+=1

    normalised_incomplete_spectrum = np.array(incomplete_spectrum)

    return normalised_incomplete_spectrum

def batch_spectrum_processing(batch, allowed_datapoints):
    processed = []

    for spectrum in batch:

        processed.append(spectrum_processing(spectrum, allowed_datapoints))
    return torch.tensor(np.array(processed))

Let's load the data from file

In [4]:
training_spectra_file = data_path / experiment_name / "training_spectra.pt"
validation_spectra_file = data_path / experiment_name / "validation_spectra.pt"
test_spectra_file = data_path / experiment_name / "test_spectra.pt"

training_oxygenations_file = data_path / experiment_name / "training_oxygenations.pt"
validation_oxygenations_file = data_path / experiment_name / "validation_oxygenations.pt"
test_oxygenations_file = data_path / experiment_name / "test_oxygenations.pt"

train_spectra_original = torch.load(training_spectra_file)
train_oxygenations_original = torch.load(training_oxygenations_file)
validation_spectra_original = torch.load(validation_spectra_file)
validation_oxygenations_original = torch.load(validation_oxygenations_file)
test_spectra_original = torch.load(test_spectra_file)
test_oxygenations_original = torch.load(test_oxygenations_file)

Now let's look at the dimensions

In [5]:
print(train_spectra_original.size())
print(train_oxygenations_original.size())
print(train_spectra_original[0])

torch.Size([134624, 41])
torch.Size([134624])
tensor([634.9278, 600.2585, 600.2339, 587.4062, 580.4452, 573.9892, 582.9027,
        597.7095, 601.8840, 641.6681, 655.6356, 704.5982, 730.0311, 739.1377,
        762.7631, 768.2003, 789.5642, 808.6349, 811.7870, 835.5294, 866.5328,
        886.8488, 918.7031, 905.1712, 913.7165, 913.7761, 913.4937, 919.7126,
        915.4688, 919.2101, 887.4873, 870.8792, 905.5049, 883.7628, 876.9416,
        888.4904, 881.3424, 888.5063, 892.4427, 879.3855, 869.1013],
       dtype=torch.float64)


In [6]:
# Zeroing out some of the spectrum data (randomly) and normalising
allowed_datapoints = [10]

train_spectra = batch_spectrum_processing(train_spectra_original, allowed_datapoints)
validation_spectra = batch_spectrum_processing(validation_spectra_original, allowed_datapoints)
test_spectra = batch_spectrum_processing(test_spectra_original, allowed_datapoints)

In [7]:
# Reshaping initial pressure spectra to fit LSTM input size
train_spectra = torch.reshape(train_spectra, (len(train_spectra), len(train_spectra[0]), 1))
validation_spectra = torch.reshape(validation_spectra, (len(validation_spectra), len(validation_spectra[0]), 1))
test_spectra = torch.reshape(test_spectra, (len(test_spectra), len(test_spectra[0]), 1))

train_oxygenations = torch.reshape(train_oxygenations_original,(len(train_oxygenations_original),1))
validation_oxygenations = torch.reshape(validation_oxygenations_original,(len(validation_oxygenations_original),1))
test_oxygenations = torch.tensor(np.float32(test_oxygenations_original))
test_oxygenations = torch.reshape(test_oxygenations_original,(len(test_oxygenations_original),1))

In [8]:
class MultiSpectralPressureO2Dataset(Dataset):
    def __init__(self, spectra, oxygenations, transform=None, target_transform=None):
        self.data = spectra
        self.labels = oxygenations
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        data = self.data[idx]
        label = self.labels[idx]
        if self.transform:
            data = self.transform(data)
        if self.target_transform:
            label = self.target_transform(label)
        return data, label

In [9]:
def switch_seq_feat(tensor):
    # Return a view of the tesor with axes rearranged
    return torch.permute(tensor, (1, 0))
training_dataset = MultiSpectralPressureO2Dataset(train_spectra, train_oxygenations, transform=switch_seq_feat)
training_dataloader = DataLoader(training_dataset, batch_size=2048, shuffle=True)
data, label = next(iter(training_dataloader))
data[0] = data[0].float()
print(data[0].size())
print(label[0].size())
print(data[0])
print(label[0])

torch.Size([1, 41])
torch.Size([1])
tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.4316,  0.0000,
          0.0000,  0.0000,  1.6534,  0.0000,  0.0000,  0.0000,  1.7502,  0.0000,
          0.0000,  0.5177,  0.0000,  0.0000, -0.3120,  0.0000, -0.4272,  0.0000,
         -0.6170,  0.0000,  0.0000, -0.7583,  0.0000,  0.0000, -1.0567,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000, -1.1817,
          0.0000]], dtype=torch.float64)
tensor([0.2782], dtype=torch.float64)


In [ ]:
# Define the LSTM for the conditioning network
lstm = nn.LSTM(
    41, # Input dimensions
    100, # No. of neurons in gate networks
    batch_first=True
)
# dummy inputs:
x = torch.randn(1, 41)
print(data[0].type())
print(x1.size())
print(x1.type())
y = lstm(x1)
print(y)

torch.DoubleTensor
torch.Size([1, 41])
torch.FloatTensor


Time to define the model... (both LSTM and INN)

In [ ]:
# Define the subnet for the invertible blocks (use the AllInOneBlack from FrEIA)
n_blocks = 10  # No. of invertible blocks in INN
def subnet(dims_in, dims_out):
    return nn.Sequential(nn.Linear(dims_in, 256), nn.ReLU(),
                         nn.Linear(256, dims_out))

# Define the GraphINN that combines the nodes
nodes = [InputNode(41, 1, name='input')]
conditions = [ConditionNode(41, 1, name='condition')]
nodes.append(Node(nodes[-1], AllInOneBlock, {"subnet_constructor":subnet}, conditions=conditions[0]))
for i in range(n_blocks):
    nodes.append(Node([nodes[-1].out0], AllInOneBlock, {"subnet_constructor":subnet}, conditions=conditions[0]))
nodes = nodes + conditions
inn = GraphINN(nodes, verbose=True)

class WrappedModel(nn.Module):
    def __init__(self, cond_network, inn):
        super().__init__()

        self.cond_network = cond_network
        self.inn = inn

    def forward(self, x):

        cond = [x, self.cond_network(x).squeeze()]

        z = self.inn(x, cond)
        zz = sum(torch.sum(o**2, dim=1) for o in z)
        jac = self.inn.jacobian(run_forward=False)

        return zz, jac

    def reverse_sample(self, z, cond):
        return self.inn(z, cond, rev=True)

model = WrappedModel(lstm, inn)

Time to define the training loop...

In [ ]:
# Defining optimizer etc
n_epochs = 10
decay_by = 0.01
weight_decay = 1e-5
betas = (0.9, 0.999)
log10_lr = -4.0                     # Log learning rate
lr = 10**log10_lr
lr_feature_net = lr                 # lr of the cond. network
params_trainable = list(filter(lambda p: p.requires_grad, model.parameters()))
gamma = decay_by**(1./n_epochs)
optim = torch.optim.Adam(params_trainable, lr=lr, betas=betas, eps=1e-6, weight_decay=weight_decay)
weight_scheduler = torch.optim.lr_scheduler.StepLR(optim, step_size=1, gamma=gamma)

In [ ]:
# a very basic training loop
for data, label in training_dataloader:
    optim.zero_grad()
    # Send data to GPU (https://stanford.edu/~shervine/blog/pytorch-how-to-generate-data-parallel)
    data, label = data.to(device), label.to(device)
    # pass to INN and get transformed variable z and log Jacobian determinant
    zz, jac = model.combined_model(data)
    # calculate the negative log-likelihood of the model with a standard normal prior
    neg_log_likeli = 0.5 * zz - jac
    loss = torch.mean(neg_log_likeli) / 41#tot_output_size
    # backpropagate and update the weights
    loss.backward()
    optim.step()